# Importing libraries and dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split as tts
import re
from gensim.models import Word2Vec

In [ ]:
df = pd.read_csv('/kaggle/input/spam-ham-dataset/spam_dataset.csv')

# Preprocessing dataset

In [ ]:
df.sample(2)

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
# df = df.drop(df.columns[0:2], axis=1)
# df = df.reset_index(drop=True)

In [ ]:
df.head()

In [ ]:
x = df.text
x.head()

In [ ]:
x.shape

In [ ]:
y = df.label_num
y.head()

In [ ]:
y.shape

# Applying stemmer

In [ ]:
pip install contractions

In [ ]:
from nltk.stem import PorterStemmer
# import contractions
ps = PorterStemmer()
from nltk.corpus import stopwords
from tqdm import tqdm
stopwords = stopwords.words('english')
stopwords.remove('no')
stopwords.remove('nor')
stopwords.remove('not')

In [ ]:
corpus = []
for i in tqdm(x):
    r = ((re.sub('[^a-zA-Z]',' ',contractions.fix(i))).lower()).split()
    corpus.append(' '.join([ps.stem(w) for w in r if not w in stopwords]))

In [ ]:
x = corpus
x[:2]

# Splitting train_test dataset

In [ ]:
xtr, xte, ytr, yte = tts(x,y,test_size=.2,random_state=0)

# word2vec

In [ ]:
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
wis=[]
for i in xtr: wis.append(i.split())

In [ ]:
model = Word2Vec(sentences=wis, vector_size=200, workers=-1)

In [ ]:
model.wv.most_similar('aqua')

In [ ]:
voc = list(model.wv.key_to_index.keys())
len(voc)

# Average word2vec

In [ ]:
def avg(sent):
    t=[]
    for s in tqdm(sent):
        c = 0
        v = np.zeros(200)
        for w in s.split():
            if w in voc:
                v += model.wv.get_vector(w)
                c += 1
        if c != 0:
            v/=c
            t.append(v)
        else:
            print(s)
    return np.array(t)

In [ ]:
xtr_ = avg(xtr)
xte_ = avg(xte)

# Apply algorithms

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
grid_params = { 'n_neighbors' : [10,20,30,40,50,60],
               'metric' : ['manhattan']}
knn=KNeighborsClassifier()
clf = RandomizedSearchCV(knn, grid_params, random_state=0,n_jobs=-1,verbose=1)
clf.fit(xtr_,ytr)
yp = cls.predict(xte_)
accuracy_score(yte, yp)

In [ ]:
cls = LogisticRegression()
cls.fit(xtr_,ytr)
yp = cls.predict(xte_)
accuracy_score(yte, yp)

In [ ]:
from sklearn.naive_bayes import GaussianNB
cls = GaussianNB()
cls.fit(xtr_,ytr)
yp = cls.predict(xte_)
accuracy_score(yte, yp)